# Reporte mensual por tienda

¡Hola, Operador! Este reporte está creado para darte la visibilidad del estatus de los usuarios de Bait de tu operación. El objetivo del mismo es dar las herramientas mínimas necesarias para que puedas crear estrategias en torno a Bait y tu operación. Recuerda que somos el Operador Móvil Virtual Número 1 en México. ¡Vamos por las ventas!


In [ ]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely import wkb
from dotenv import load_dotenv
import awswrangler as wr
import os
import yaml
import boto3
from IPython import display as ICD

In [ ]:
# Configs
# load environment variables with
load_dotenv()

# import config
with open("../config.yaml") as f:
    config = yaml.safe_load(f)

# connect to AWS with credentials
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION = os.getenv("REGION")
BUCKET = os.getenv("BUCKET")
FOLDER = config["aws"]["folder"]

# connect to AWS
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)
s3 = session.client("s3")

In [ ]:
# S1: Get User Data
# get users data from S3
gdf_users = (
    wr.s3.read_parquet(
        f"s3://{BUCKET}/{FOLDER}/{config['aws']['users-file']}"
    )
    .rename(columns={"client_latitude": "latitude", "client_longitude": "longitude"})
    # convert to GeoDataFrame
    .pipe(gpd.GeoDataFrame)
    .assign(
        geometry=lambda x: gpd.points_from_xy(x['longitude'], x['latitude'])
    )
    .set_crs(epsg=4326)
)

# S2: Wrangle Data
# eliminate invalid values of raw_sim_operator_name
non_valid_operators = config['report']['non-valid-operators']
gdf_users = gdf_users[~gdf_users["raw_sim_operator_name"].isin(non_valid_operators)]

# standarize the name of Megacable
gdf_users.loc[
    gdf_users["raw_sim_operator_name"].isin(["Mega 4.5G", "Mega4.5G"]), "raw_sim_operator_name"
    ] = "Megacable"

In [ ]:
# S1: Get Shape Data
# download shapefile of states
gdf_states = (
    wr.s3.read_parquet(
        f"s3://{BUCKET}/{FOLDER}/{config['aws']['states-file']}"
    )
    # pass geometry column from binary to geometry
    .assign(geometry=lambda x: x['geometry'].apply(wkb.loads))
    .pipe(gpd.GeoDataFrame)
    .set_crs(epsg=4326)
    .rename(columns={"CODIGO": "cve_ent", "ESTADO": "cve_name"})
    .assign(
        cve_ent=lambda x: x['cve_ent'].str[-2:],
        cve_name=lambda x: x['cve_name'].str.lower()
    )
    .sort_values('cve_ent', ignore_index=True)
)

# S2: Wrangle Data
# get data of walmart stores
re_walmart = r"(walmart|wal mart|superama|waltmart)"
re_sams = r"(sams|sam's|sam s|sam's club|sam s club|sam'sclub|sam sclub|sam club|mi bodega)"
re_bodega = r"(bodega aurrera|bodega|aurrera|ba|boa|\$b|mb|b )"
re_supercenter = r"(supercenter|super center)"

# get walmart stores
gdf_walmart = (
    wr.s3.read_csv(
        f"s3://{BUCKET}/{FOLDER}/{config['aws']['walmart-file']}"
    )
    .assign(
        geometry=lambda x: gpd.points_from_xy(x['longitude'], x['latitude']),
        # get bodega aurrera or walmart or sams in name
        store_name=lambda x: np.select(
            [
                x['name'].str.contains(re_bodega, case=False),
                x['name'].str.contains(re_walmart, case=False),
                x['name'].str.contains(re_sams, case=False),
                x['name'].str.contains(re_supercenter, case=False)
            ],
            ['bodega aurrera', 'walmart', 'sams', 'supercenter'],
            default='other'
        )
    )
    .pipe(gpd.GeoDataFrame, crs="EPSG:4326")
    .query("store_name != 'other'")
    .loc[:, [
             'id', 'store_name', 'name', 'staff_stratum_description',
             'postal_code', 'cve_ent', 'cve_mun', 'geometry'
            ]]
)

In [ ]:
# S1: Get Connections Data
# get connections data from S3
gdf_connections = (
    wr.s3.read_parquet(
        f"s3://{BUCKET}/{FOLDER}/{config['aws']['connections-file']}"
    )
)

# S2: Wrangle Data
# obtain the day of the week of each test
gdf_connections['day_of_week'] = gdf_connections['result_date'].dt.day_name()

# Translate the days to Spanish
def days_to_Spanish(argument):
    days = {
        "Monday": "Lunes",
        "Tuesday": "Martes",
        "Wednesday": "Miércoles",
        "Thursday": "Jueves",
        "Friday": "Viernes",
        "Saturday": "Sábado",
        "Sunday": "Domingo"
    }
    return days.get(argument, "error")

gdf_connections['day_of_week'] = gdf_connections['day_of_week'].apply(days_to_Spanish)

# Participación de mercado

Zonas Nielsen:

-   BAJÍO: Aguascalientes, Jalisco, Guanajuato, Colima, Michoacán

-   PACÍFICO: Baja California, Baja California Sur, Sinaloa, Sonora, Nayarit

-   NORTE: Chihuahua, Coahuila, Durango, Nuevo León, San Luis Potosí, Tamaulipas, Zacatecas

-   SURESTE: Campeche, Chiapas, Oaxaca, Quintana Roo, Tabasco, Veracruz, Yucatán

-   CENTRO: CDMX, Guerrero, Hidalgo, EDOMEX, Morelos, Puebla, Querétaro, Tlaxcala


In [ ]:
# Join Data
# join users with states
tbl_users_state = (
    gpd.sjoin_nearest(
        gdf_users.to_crs("EPSG:6372"), gdf_states.to_crs("EPSG:6372"),
    )
    .drop_duplicates(subset=["device_id"])
    .filter(["device_id", "postal_code", "raw_sim_operator_name", "cve_name"])
)

# define the dictionary
nielsen_zones = config['report']['nielsen_zones']

# map the states to their corresponding regions
tbl_users_state['region'] = (
    tbl_users_state['cve_name']
    .map(
        lambda x: next((region for region, states in nielsen_zones.items() if x.lower() in states), None)
        )
)

# droup by 'raw_sim_operator_name' and count 'device_id', then sort and get the top 10
top_operators = (
    tbl_users_state
    .groupby(["raw_sim_operator_name"])['device_id']
    .count()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
    ["raw_sim_operator_name"]
)

# Wrangle
tbl_users_state.loc[~tbl_users_state["raw_sim_operator_name"].isin(top_operators), "raw_sim_operator_name"] = "other"
# group by 'raw_sim_operator_name' and count 'device_id'
df_top_operator = tbl_users_state.groupby(["raw_sim_operator_name", "region"])['device_id'].count().reset_index()
# Get market share
df_top_operator["market_share"] = df_top_operator.groupby("region")["device_id"].transform(lambda x: x / x.sum())
# Pivot the DataFrame for the stacked bar chart
pivot_df = df_top_operator.pivot(index='region', columns='raw_sim_operator_name', values='market_share')
# Fill NaN values with 0
pivot_df = pivot_df.fillna(0)

# Filter the market share of bait in each region and select the maximum and minimum
bait_share = df_top_operator[df_top_operator["raw_sim_operator_name"]=="BAIT"]
# Maximum and minimum share
max_p = np.max(bait_share.market_share)
min_p = np.min(bait_share.market_share)
# Obtain the best and worst region based on the share market
best_region = bait_share[bait_share["market_share"]==max_p].region.iloc[0]
worst_region = bait_share[bait_share["market_share"]==min_p].region.iloc[0]
best_region_p = round(max_p*100,2)
worst_region_p = round(min_p*100,2)


# Plot
# stacked bar chart
pivot_df.plot(kind='bar', stacked=True, figsize=(12, 8), colormap='tab20')
# add labels and title
plt.xlabel('Estado')
plt.ylabel('Device ID Count')
plt.title('Top 10 SIM Operators by Device ID Count per Estado')
plt.legend(title='SIM Operator Name', bbox_to_anchor=(1.05, 1), loc='upper left')
# y lim
plt.ylim(0, 1)
# y in percentage
plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()])
# display
plt.tight_layout()
plt.show()

Tomando en cuenta a todos los Operadores Móviles Virtuales (OMVs) de la red de ALTÁN, podemos visualizar la participación de Bait vs los otros operadores del mercado. Es evidente que en todas las zonas del país Bait es el OMV número 1. La zona con mayor participación es `{python} best_region` con un `{python} best_region_p`%, por otro lado, la zona con menor participación es `{python} worst_region` con un `{python} worst_region_p`%. Con esto, se recomienda replicar la estrategia de venta que está generado la zona `{python} best_region` en `{python} worst_region` .

Por otro lado, a nivel nacional los siguientes 4 OMVs con mayor participación de mercado son `{python} top_operators[1]`, `{python} top_operators[2]`, `{python} top_operators[3]` y `{python} top_operators[4]`. Es imperante entender qué están haciendo estos operadores en las zonas que están teniendo más penetración para que Bait mantenga su posición en el mercado.

# Presencia nacional

En el siguiente mapa se puede observar la presencia nacional de Bait en cada una de las tiendas (puntos de venta). El color de la burbuja nos indica el número de clientes Bait por tienda. Para decidir si un cliente Bait es de una tienda en específico a cada conexión registrada en la base de datos se le asigna la tienda más cercana. Posteriormente, a cada usuario registrado, se le asigna la tienda donde se hayan registrado el mayor número de conexiones.


In [ ]:
# Select only BAIT users
gdf_bait_users = gdf_users[gdf_users["raw_sim_operator_name"] == "BAIT"]

# Join users with stores getting the closest store
gdf_clients_stores = (
    gpd.sjoin_nearest(
        gdf_bait_users.to_crs("EPSG:6372"), gdf_walmart.to_crs("EPSG:6372"),
    )
    .drop_duplicates(subset=["device_id"])
    .groupby(["name", "store_name", "id", "cve_ent", "cve_mun"])
    .agg(
        count=("device_id", "count")
    )
    .reset_index()
    .sort_values("count", ascending=False)
    .merge(
        gdf_walmart.loc[:, ["id", "geometry"]],
        on="id",
        how="left"
    )
    .pipe(gpd.GeoDataFrame)
    .set_crs(epsg=4326)
    .assign(
        index=lambda x: (x['count'] - x['count'].min()) / (x['count'].max() - x['count'].min())
    )
)

# plot states with count of users
fig, ax = plt.subplots(figsize=(12, 6))
gdf_states.plot(ax=ax, color='lightgray', edgecolor='black')
gdf_clients_stores.plot(
    ax=ax,
    markersize=1.5,
    column='index',
    cmap='magma_r',
    legend=True,
    alpha=1
)
# title
plt.title('Concentración de usuarios Bait por tienda')
# x and y ticks off
plt.xticks([])
plt.yticks([])
# legend more readable, loc in bottom 

# display
plt.show()

Como se puede observar el centro del país es donde se tiene un mayor número de clientes Bait, lo cual está relacionado a la cantidad de unidades en la región. Lo anterior nos confirma la alta dependencia que tenemos de los puntos de venta físicos para seguir generando nuevos usuarios, le sigue la región sureste y por último la región norte, la cual tiene el menor número de clientes activos Bait.

# Análisis de tiendas por region


In [ ]:
bodega_top = (gdf_clients_stores[gdf_clients_stores["store_name"]=="bodega aurrera"]
    .sort_values("count", ascending=False)
    .name
    .iloc[0]
)

walmart_top = (gdf_clients_stores[(gdf_clients_stores["store_name"]=="supercenter") |
    (gdf_clients_stores["store_name"]=="walmart")]
    .sort_values("count", ascending=False)
    .name
    .iloc[0]
)

sams_top = (gdf_clients_stores[gdf_clients_stores["store_name"]=="sams"]
    .sort_values("count", ascending=False)
    .name
    .iloc[0]
)

Es importante replicar las buenas prácticas que están realizando las tiendas con mayor número de usuarios, asimismo, es imperante que las tiendas con menor número de usuarios ejecuten nuevas estrategias para no distanciarse más de las mejores. A nivel nacional las tiendas top por formato son:

-   Bodega Aurrera: `{python} bodega_top`
-   Walmart Supercenter: `{python} walmart_top`
-   Sams: `{python} sams_top`

A continuación, se verán las 5 tiendas Top vs 5 tiendas Bottom de cada una de las Zonas Nielsen del país:


In [ ]:
# join users with states
gdf_clients_stores_region = (
    gpd.sjoin_nearest(
        gdf_clients_stores.to_crs("EPSG:6372"), gdf_states.to_crs("EPSG:6372"),
    )
)

# Map the states to their corresponding regions
gdf_clients_stores_region['region'] = (
    gdf_clients_stores_region['cve_name']
    .map(lambda x: 
    next((region for region, states in nielsen_zones.items() if x.lower() in states),None))
)

# Function to print the top 5 and bottom 5 per region
def top_bottom(region):
    stores = (gdf_clients_stores_region[gdf_clients_stores_region["region"]==region]
        .sort_values("count", ascending=False)
        )[["name", "store_name","cve_name","count"]]
    stores.columns = ["Nombre", "Formato", "Estado", "Clientes_reales"]

    ICD.display(f"Top 5 tiendas {region}")
    ICD.display(stores.head(n=5))
    ICD.display(f"Bottom 5 tiendas {region}")
    ICD.display(stores.tail(n=5))

for zone in nielsen_zones:
    top_bottom(zone)
    print("\n")

# Análisis temporal

# Distribución de usuarios por día de la semana


In [ ]:
# Obtain the number of users per day of the week
potential_users = (
    gdf_connections
    .groupby("day_of_week")
    .agg(
        count=("device_id", "count")
    )
    .reset_index()
)
# Sort the rows by day of the week
sorted_weekdays = config['report']['weekdays']
potential_users['day_of_week'] = pd.Categorical(potential_users['day_of_week'], sorted_weekdays)
potential_users = potential_users.sort_values("day_of_week")

# Filter the day with more traffic
max_traffic = max(potential_users["count"])
best_day = (
    potential_users[potential_users["count"]== max_traffic]
    .day_of_week
    .iloc[0]
)

# Filter the day with less traffic
less_traffic = min(potential_users["count"])
worst_day = (
    potential_users[potential_users["count"]== less_traffic]
    .day_of_week
    .iloc[0]
)

# Obtain the number of BAIT users per day of the week
df_real_users = (
    gdf_connections
    [gdf_connections["raw_sim_operator_name"] == "BAIT"]
    .groupby("day_of_week")
    .agg(
        count=("device_id", "count")
    )
    .reset_index()
)
# Sort the rows by day of the week
df_real_users['day_of_week'] = pd.Categorical(df_real_users['day_of_week'], sorted_weekdays)
df_real_users = df_real_users.sort_values("day_of_week")

# Plot
fig, axes = plt.subplots(2, 1, figsize=(6, 6))
# First plot
axes[0].bar(potential_users['day_of_week'], potential_users['count'])
axes[0].set_title('Número de usuarios por día de la semana')

# Second plot
axes[1].bar(df_real_users['day_of_week'], df_real_users['count'])
axes[1].set_title('Número de usuarios BAIT por día de la semana')

# Adjust layout
plt.tight_layout()
# Display
plt.show()

Debido a que se tienen todas las conexiones de los usuarios de todos los OMVs, podemos identificar clientes potenciales para cada tienda con base en el número de usuarios que están activos en el mismo código postal de la unidad. Por ello, podemos identificar qué día de la semana hay más clientes potenciales vs el día con menor número. Por otro lado, se puede visualizar el mismo comportamiento, pero con el número de clientes Bait. Al realizar este contraste, podemos ver que el día que más usuarios potenciales están cercanos a la unidad es el `{python} best_day`, por el contrario, el día con menos usuarios potenciales, en el cual se podrían crear distintas estrategias para incentivar la presencia de más clientes es el `{python} worst_day`.


In [ ]:
# Obtain the number of BAIT clients per store based in their postal_code
df_bait_users_per_store = (
    gdf_clients_stores_region
    .sort_values("count", ascending=False)
    [["name", "store_name","cve_name","count"]]
    )

# columns in spanish for better understanding
df_bait_users_per_store.columns = ["Nombre", "Formato", "Estado", "Clientes_reales"]

# Obtain the postal code of the most and less crowded store
top_postal_code = str(gdf_walmart[gdf_walmart["name"] == df_bait_users_per_store.head(n=1)["Nombre"][0]].postal_code.iloc[0])
bottom_postal_code = str(gdf_walmart[gdf_walmart["name"] == df_bait_users_per_store.tail(n=1)["Nombre"].iloc[0]].postal_code.iloc[0])